# Import nessacary libraries

In [1]:
# %pip install selenium
# %pip install pymongo
# %pip install beautifulsoup4

In [2]:
# import some basic python libaries (operating system, regular expressions, time)
import os
import re
import time
from datetime import datetime
import random
import requests
# import some selenium components to rememeneber by name
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException

# Import Load login credentials from .env file, make sure to pip install python-dotenv
#import mongoDB, we may add additional databases in the future for better 

from pymongo import MongoClient

from bs4 import BeautifulSoup

#import custom library 
import fun_functions as ff

# Setup 

In [3]:

# Create a MongoClient to the running mongod instance
client = MongoClient("mongodb://localhost:27017")
db = client["my_database"]
#Delete all documents in the collection before we start 
collection_levels_fyi = db['opportunities_levels_fyi']
result = collection_levels_fyi.delete_many({})
print(result.deleted_count, " documents deleted.")
url = r"https://www.levels.fyi/internships/?track=Software%20Engineer&timeframe=Summer%20-%202024"

#current tie for log name
timestamp = datetime.now().strftime('%Y_%m_%d_%H-%M-%S')

0  documents deleted.


# Data Ingestion

In [4]:
#get all jobs and store them in a list of objects in mongo db
driver = webdriver.Chrome()
driver.get(url)
time.sleep(5)

job_description_section = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "internships-table")))
job_box_html = job_description_section.get_attribute('outerHTML')
# print (job_box_html)


#response = requests.get(url)
# soup = BeautifulSoup(response, 'html.parser')

driver.quit()


In [5]:
# Now we use soup to parse the html and get the data we need
test = requests.get(url, timeout= 5)
soup = BeautifulSoup(job_box_html, 'html.parser')
all_data = []

rows = soup.find_all('tr')
filtered_rows = []
tag = soup.a
for element in rows:
    if "Not Open" in element.text:
        continue
    else:
        filtered_rows.append(element)

print(len(filtered_rows))
# g
for item in filtered_rows:
    data = {}
    smaller_soup = BeautifulSoup(str(item), 'html.parser')
    # extract the data from the html and put into documents to deal with later

        #   GET THE LINK
    link = smaller_soup.find("a")
    if link and 'href' in link.attrs:
        href = link['href']
        link_text = link.text.strip()
       # print(f"Link: {href}")
        # print(href)
        #print(f"Text: {link_text}")
        # Only keep applying if the link look valid
        if "Apply" in link_text:
            data["href"] = href
        else:
            data["href"] = "NA"
            
    else:
        print("No <a> tag found in this element.")
        continue

        #    GET THE SALARY
    salary = smaller_soup.find("div",class_= "salary-info")
    if salary:
        # print(salary)
        # print(salary.text)
        cleaned_salary_text = salary.text.rstrip().lstrip().replace("\n", "").replace("Apply", "").replace('Add Compensation',"NA")
        # print(cleaned_salary_text)
        data["salary"] = cleaned_salary_text

        #save info to object
    else:
        print("No salary found in this element.")


        #   GET THE LOCATION AND COMPANY
    location = smaller_soup.find("div", class_="media-body align-self-center")
    if location:
        # Split the text by newlines and strip any leading/trailing whitespace
        cleaned_parts = [part.strip() for part in location.text.split('\n') if part.strip()]
        company_name = cleaned_parts[0]  #company i think
        location = cleaned_parts[1] #loction i think

        
        
        
        if "-" in location:
            location_and_time_list = location.split("-")
            real_location = location_and_time_list[0]
            season = location_and_time_list[1]
            data["location"] = real_location
            data["season"] = season
        else: 
            data["location"] = location

        # print (real_location)
        # print (season)

        data["company"] = company_name
        #
    else:
        print("No location found in this element.")

        #   GET THE Details, we can parse the link with ai later to get a description if we want to explore that route.
    details = smaller_soup.find("td", class_="tags-th lfont m-3 py-3")
    if details:
        cleaned_details = details.text
        # print(cleaned_details)
        data['details'] = cleaned_details
    else:
        print("No details found in this element.")

    # data = {"details": details, "date": datetime.now()}
    # print(data)
    
        # Try to insert the document
    collection_levels_fyi.insert_one(data)
    print(f"{data} successfully inserted into the database.")
    
    # print(smaller_soup.prettify)
    #print("\n\n")

# print(filtered_rows.text)
# all_jobs = soup.find_all( class_="media company-info-cell")
# print(len(all_jobs))
# print(all_jobs[0].prettify())


97
No <a> tag found in this element.
No <a> tag found in this element.
{'href': 'https://www.deshaw.com/careers/software-developer-intern-new-york-summer-2024-4803?ref=levels.fyi', 'salary': 'CA$172.35 / hrCA$29,875 / mo', 'location': 'New York City, NY ', 'season': '  Summer / 2024', 'company': 'D.E. Shaw', 'details': '\nPhD\nCorporate housing\n\nQuantitative Researcher\n\n', '_id': ObjectId('666d0b7110137067ce426f90')} successfully inserted into the database.
{'href': 'https://optiver.com/working-at-optiver/career-opportunities/?_gl=1*rzd3uv*_up*MQ..&gclid=CjwKCAjw-7OlBhB8EiwAnoOEk9DE1HiDcs8Rzl5UWzLbMcHX2U_phUQrWHrl4A-soUm_cw8fMfCC3BoCcBcQAvD_BwE&numberposts=10&paged=1&search=intern&ref=levels.fyi', 'salary': 'CA$170 / hrCA$29,423 / mo', 'location': 'Chicago, IL ', 'season': '  Summer / 2024', 'company': 'Optiver', 'details': '\nUndergrad (Junior)\nCorporate housing, company provided transportation and relocation, CA$13,580 sign-on bonus\n\nQuantitative Trader\n\n', '_id': ObjectId('

In [6]:
# print(soup.prettify())
